In [1]:
import math
import torch

In [2]:
torch.set_grad_enabled(False)

In [3]:
# This will be our main class from which a lot of other classes will inherit (e.g. fully connected layer)
class Module(object):
    def __init__(self):
        self.output = None
        self.grad_input = None
        self.train = True
    
    def forward(self, inpt):
        return self.update_output(inpt)

    def backward(self, inpt, grad_output):
        self.update_grad_input(inpt, grad_output)
        self.acc_grad_params(inpt, grad_output)
        return self.grad_input
    
    def update_output(self, inpt):        
        pass

    # gradient with respect to input
    def update_grad_input(self, inpt, grad_output):       
        pass   
    
    # gradient with respect to parameters
    def acc_grad_params(self, inpt, grad_output):
        pass
    
    def zero_grad_params(self): 
        pass
        
    def get_params(self):
        return []
        
    def get_grad_params(self):
        return []

In [4]:
# this is a container class like keras's Sequential model
class Sequential(Module):    
    def __init__ (self):
        super(Sequential, self).__init__()
        self.modules = []
        self.y = []
   
    def add(self, module):
        self.modules.append(module)

    def update_output(self, inpt): 
        self.y = [] 

        y_p = self.modules[0].forward(inpt)
        self.y.append(y_p)
        for i in range(1,len(self.modules)):
            y_n = self.modules[i].forward(y_p)
            self.y.append(y_n)
            y_p = y_n
        
        self.output = y_n
            
        return self.output

    def backward(self, inpt, grad_output): 
        n = len(self.modules)
        g_n = self.modules[n-1].backward(self.y[n-2],grad_output)
        for i in range(n-2,0,-1):
            g_p = self.modules[i].backward(self.y[i-1],g_n)
            g_n = g_p
            
        self.grad_input = self.modules[0].backward(inpt,g_p)
        return self.grad_input

    def zero_grad_params(self): 
        for module in self.modules:
            module.zero_grad_params()
    
    def get_params(self):
        return [x.get_params() for x in self.modules]
    
    def get_grad_params(self):
        return [x.get_grad_params() for x in self.modules]

In [5]:
# input: batch_size x n_features1
# output: batch_size x n_features2

class DenseLayer(Module):
    def __init__(self, n_in, n_out):
        super(DenseLayer, self).__init__()
       
        #initializing weights 
        stdv = 1./math.sqrt(n_in)
        self.W = torch.FloatTensor(n_out,n_in).uniform_(-stdv,stdv)
        self.b = torch.FloatTensor(n_out).uniform_(-stdv,stdv)
        
        self.gradW = torch.zeros_like(self.W)
        self.gradb = torch.zeros_like(self.b)
    
    def update_output(self, inpt):
        self.output = torch.mm(inpt,self.W.T) + self.b  
        return self.output

    def update_grad_input(self, inpt, grad_output):
        self.grad_input = torch.mm(grad_output,self.W)
        return self.grad_input
    
    def acc_grad_params(self, inpt, grad_output):
        self.gradW = torch.mm(grad_output,input)
        self.gradb = grad_output.sum(axis=0) 
    
    def zero_grad_params(self):
        self.gradW = torch.zeros_like(self.gradW)
        self.gradb = torch.zeros_like(self.gradb)
        
    def get_params(self):
        return [self.W, self.b]
    
    def get_grad_params(self):
        return [self.gradW, self.gradb]

In [6]:
d_layer = DenseLayer(2, 30)

In [7]:
d_layer.get_grad_params()

[tensor([[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]]),
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.])]

In [8]:
d_layer.get_params()

[tensor([[ 0.4079,  0.2833],
         [-0.6016, -0.5495],
         [-0.4683, -0.6694],
         [ 0.2851,  0.0040],
         [ 0.4665,  0.3928],
         [ 0.6579,  0.1451],
         [-0.1126,  0.5906],
         [ 0.0165,  0.0917],
         [ 0.2582, -0.3816],
         [-0.4919,  0.2812],
         [-0.2568,  0.3943],
         [-0.5193, -0.2884],
         [-0.6625, -0.0255],
         [-0.5337, -0.2044],
         [-0.6906,  0.0825],
         [-0.7061, -0.5512],
         [-0.0746, -0.1444],
         [ 0.4210,  0.4040],
         [ 0.0886, -0.0594],
         [-0.3850,  0.1256],
         [ 0.5332, -0.4738],
         [-0.1632, -0.5522],
         [ 0.6804,  0.0813],
         [ 0.4823,  0.1149],
         [ 0.1545, -0.4765],
         [ 0.6300,  0.1906],
         [-0.4823, -0.3331],
         [ 0.1652, -0.2538],
         [-0.4403,  0.5474],
         [ 0.5304, -0.6658]]),
 tensor([ 0.2682,  0.2498,  0.5760,  0.3510, -0.5641,  0.2335,  0.2658, -0.1946,
          0.1685, -0.5998,  0.5618, -0.6468, -0

In [9]:
class Tanh(Module):
    def __init__(self):
         super(Tanh, self).__init__()
    
    def update_output(self, inpt):
        self.output = torch.tanh(inpt)
        return self.output
    
    def update_grad_input(self, inpt, grad_output):
        self.grad_input = (1 - self.output**2)*grad_output
        return self.grad_input
    
    def __repr__(self):
        return "Tanh"

In [ ]:
# EXPERIMENTING

# class Animal(object):
#     def __init__(self,domestic,colour):
#         self.name = None
#         self.domestic = domestic
#         self.colour = colour
    
#     def sound(self):
#         pass
    
#     def reality(self):
#         print('Humans are more clever than any other animal on the planet')
        
#     def __repr__(self):
#         return "Module"
        

# class Haski(Animal):
#     def __init__(self, domestic, colour, name):
#         super().__init__(domestic,colour)
#         self.name = name
        
#     def sound(self):
#         print('Haf-Haf')
        
#     def __repr__(self):
#         return "Dog"

In [ ]:
# my_dog = Haski(True,'White','Leo')
# my_dog.reality()